# Create AMITT website sql

Creates the sqlite file used as a datasource in the AMITT website tests. 

In [2]:
# Get AMITT variables

import pandas as pd
import sqlite3 as sql
import os
from sqlalchemy import create_engine
from generate_amitt_ttps import Amitt
pd.set_option('display.max_rows', 1000)


# Generate AMITT datasets
amitt = Amitt()

dbasetype = 'postgresql_local' #'sqlite' #'heroku'


# Create connection to database
if dbasetype == 'sqlite':
    # Generate minimal sqlite database from the Amitt variables
    conn = sql.connect('amittsite.sqlite')
elif dbasetype == 'postgresql_local':
    conn = create_engine('postgresql://sara:@localhost:5432/amittsite')

    
# Add table to database    
def add_table(dataframe, tablename, columns):    
    # Create sql-appropriate table from dataframe
    colnames = ', '.join(['{} TEXT NOT NULL'.format(col) for col in columns])
    newtable = dataframe[columns].copy().applymap(str)
    newtable['id'] = range(1,len(newtable)+1)
    
    # send to database
    if dbasetype == 'sqlite':
        conn.execute("DROP TABLE IF EXISTS {}".format(tablename))
        conn.execute('''CREATE TABLE {} (id INTEGER PRIMARY KEY AUTOINCREMENT, {});'''.format(tablename, colnames))
        newtable.to_sql(tablename, conn, index=False, if_exists='append')
        conn.commit()
    elif dbasetype == 'postgresql_local':
        newtable.to_sql(tablename, conn, index=False, if_exists='replace')
    return newtable


# Build a cross-table
def object_tactics_techniques(objectcol, objecttable, crosstable):
    # objects to techniques
    ctech = crosstable.copy()
    ctech = ctech[(ctech['technique_id'] != '') & (~ctech['technique_id'].str.startswith('TA'))]
    ctech.rename(columns={'amitt_id':objectcol}, inplace=True)
    ctech['summary'] = 'N/A'

    # objects to tactics
    ctact = crosstable[crosstable['technique_id'].str.startswith('TA')].copy()
    ctact.rename(columns={'amitt_id':objectcol, 'technique_id': 'tactic_id'}, inplace=True)
    ctact['main_tactic'] = 'N'
    ctactmain = objecttable[['amitt_id', 'tactic_id']].copy()
    ctactmain.rename(columns={'amitt_id':objectcol}, inplace=True)
    ctactmain['main_tactic'] = 'Y'
    ctact = pd.concat([ctact, ctactmain], ignore_index=True, sort=False)
    ctact['summary'] = 'N/A'
    return(ctech, ctact)

#Load all the tables - Heroku needs them in correct order... 

# -- frameworks --

#newtable = add_table(amitt.df_actortypes, 'actor_type', ['amitt_id', 'sector_id', 'framework_id', 'name', 'summary'])
newtable = add_table(amitt.df_counters, 'counter', ['amitt_id', 'tactic_id', 'metatechnique_id', 'name', 'summary'])
newtable = add_table(amitt.df_detections, 'detection', ['amitt_id', 'tactic_id', 'name', 'summary'])
newtable = add_table(amitt.df_frameworks, 'framework', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_metatechniques, 'metatechnique', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_phases, 'phase', ['amitt_id', 'name', 'rank', 'summary'])
#newtable = add_table(amitt.df_playbook, 'playbook', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_resources, 'resource', ['amitt_id', 'name', 'summary', 'resource_type'])
newtable = add_table(amitt.df_responsetypes, 'responsetype', ['amitt_id', 'name', 'summary'])
#newtable = add_table(amitt.df_sector, 'sector', ['amitt_id', 'name', 'summary'])
newtable = add_table(amitt.df_tactics, 'tactic', ['amitt_id', 'phase_id', 'name', 'rank', 'summary'])
newtable = add_table(amitt.df_tasks, 'task', ['amitt_id', 'tactic_id', 'framework_id', 'name', 'summary'])
newtable = add_table(amitt.df_techniques, 'technique', ['amitt_id', 'tactic_id', 'name', 'summary'])

(ctech, ctact) = object_tactics_techniques('counter_id', amitt.df_counters, amitt.cross_counterid_techniqueid)
newtable = add_table(ctech, 'counter_technique', ['counter_id', 'technique_id', 'summary'])
newtable = add_table(ctact, 'counter_tactic', ['counter_id', 'tactic_id', 'main_tactic', 'summary'])

(dtech, dtact) = object_tactics_techniques('detection_id', amitt.df_detections, amitt.cross_detectionid_techniqueid)
newtable = add_table(dtech, 'detection_technique', ['detection_id', 'technique_id', 'summary'])
newtable = add_table(dtact, 'detection_tactic', ['detection_id', 'tactic_id', 'main_tactic', 'summary'])

# -- datasets --

# dataset
# reference - create this from other tables. 
# incidenttechnique crosstable - and others, inc incidentcounter - create from data
newtable = add_table(amitt.df_groups, 'group', ['amitt_id', 'name', 'url', 'summary', 
                                                'sector', 'primary_role', 'secondary_role', 
                                                'primary_subject', 'secondary_subject', 
                                                'volunteers', 'region', 'country', 
                                                'twitter_handle'])
newtable = add_table(amitt.df_incidents, 'incident', ['amitt_id', 'name', 'summary', 
                                                      'year_started', 'attributions_seen', 
                                                      'found_in_country', 'objecttype'])
newtable = add_table(amitt.df_tools, 'tool', ['amitt_id', 'name', 'summary',
                                              'organization', 'url', 'category', 
                                              'disinformation_use', 'cogseccollab_use', 
                                              'function', 'code_url', 'artifacts', 
                                              'automation', 'platform', 'accessibility'])

# Load in users table, and close connection
if dbasetype == 'sqlite':
    conn.execute("DROP TABLE IF EXISTS {}".format('users'))
    conn.execute('''CREATE TABLE users (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT NOT NULL UNIQUE, password TEXT NOT NULL);''')
    conn.close()
else:
    dfusers = pd.DataFrame([['test','testing']], columns=['username', 'password'])
    usertable = add_table(dfusers, 'users', ['username', 'password'])

newtable

,amitt_id,name,summary,organization,url,category,disinformation_use,cogseccollab_use,function,code_url,artifacts,automation,platform,accessibility,id
0,TL00001,Account Analysis,,Account Analysis,https://accountanalysis.app/,Social Media Search and Account Analysis,,,Account Analysis is another great Twitter acco...,,,,,,1
1,TL00002,Ad Observer,,New York University,https://chrome.google.com/webstore/detail/ad-o...,Advertising Analysis,Designed for disinfo,,Collect and share ads,,ads,human,browser extension,,2
2,TL00003,Aletheia,,Midstream Technology,http://midstream.us,Network analysis,Designed for disinfo,,social network analysis; astroturf,,accounts,,,,3
3,TL00004,Alexa,,Alexa,https://www.alexa.com,Web Traffic Analysis,,,,,,,,,4
4,TL00005,AnalyzeID,,,http://analyzeid.com/,Website Analysis,Useful,,AnalyzeID find related websites through ad codes,,,,,,5
5,TL00006,Archive.is,,,http://archive.is/,Website Analysis,,,Archive.is often used for social media profile...,,,,,,6
6,TL00007,Assembler,,,https://projectassembler.org/,Image analysis,Designed for disinfo,,image analysis,,images,,,,7
7,TL00008,Backlink Watch,,Backlink Watch,http://www.backlinkwatch.com/,Web Traffic Analysis,,,Backlink checker,,,,,,8
8,TL00009,Blackbird Constellation Engine,,Blackbird,https://www.blackbird.ai/,,Designed for disinfo,,,,,,,paid,9
9,TL00010,Bot Sentinel,,,https://botsentinel.com/,Bot analysis,Designed for disinfo,yes,,,accounts,,,,10


## test area

In [3]:
# Check which amitt variables we can see from here
print('{}\n'.format(vars(amitt).keys()))
testdfname = 'df_incidents'
testdf = vars(amitt)[testdfname]
print('{} columns: {}'.format(testdfname, testdf.columns))
testdf

dict_keys(['df_phases', 'df_frameworks', 'df_techniques', 'df_tasks', 'df_incidents', 'df_groups', 'df_tools', 'df_counters', 'df_detections', 'df_actortypes', 'df_resources', 'df_responsetypes', 'df_metatechniques', 'it', 'df_tactics', 'df_techniques_per_tactic', 'df_counters_per_tactic', 'phases', 'tactics', 'techniques', 'counters', 'metatechniques', 'actortypes', 'resources', 'num_tactics', 'cross_counterid_techniqueid', 'cross_counterid_resourceid', 'cross_counterid_actortypeid', 'cross_detectionid_techniqueid', 'cross_detectionid_resourceid', 'cross_detectionid_actortypeid'])

df_incidents columns: Index(['amitt_id', 'name', 'objecttype', 'summary', 'year_started',
       'attributions_seen', 'found_in_country', 'urls', 'notes', 'when_added',
       'found_via', 'longname'],
      dtype='object')


,amitt_id,name,objecttype,summary,year_started,attributions_seen,found_in_country,urls,notes,when_added,found_via,longname
0,I00001,Blacktivists facebook group,incident,Internet Research Agency created fake @blackti...,2016,Russia,USA,https://money.cnn.com/2017/09/28/media/blackti...,,2019-02-24,,I00001 - Blacktivists facebook group
1,I00002,#VaccinateUS,campaign,use both pro- and anti- topic messaging to cre...,2014,Russia,World,https://www.washingtonpost.com/science/2018/08...,,2019-02-24,,I00002 - #VaccinateUS
2,I00003,Beyonce protest rallies,incident,use both pro- and anti- topic messaging to cre...,2016,Russia,USA,https://www.theguardian.com/us-news/2018/may/1...,,2019-02-24,,I00003 - Beyonce protest rallies
3,I00004,#Macrongate,incident,amplified document dump failed because France ...,2017,Russia,France,http://www.niemanlab.org/2018/09/how-france-be...,,2019-02-24,,I00004 - #Macrongate
4,I00005,Brexit vote,campaign,"In early 2014, then UK PM David Cameron outlin...",2016,Russia,UK,https://www.forbes.com/sites/emmawoollacott/20...,,2019-02-24,,I00005 - Brexit vote
5,I00006,Columbian Chemicals,incident,Early Russian (IRA) “fake news” stories. Compl...,2014,Russia,USA,https://en.wikipedia.org/wiki/Columbian_Chemic...,,2019-02-24,,I00006 - Columbian Chemicals
6,I00007,Incirlik terrorists,incident,Fake story transmitted from Russian media to T...,2016,Russia,USA,https://www.politifact.com/truth-o-meter/state...,,2019-02-24,,I00007 - Incirlik terrorists
7,I00008,Bujic,incident,,2017,Russia,Serbia,https://www.stopfake.org/en/vencislav-the-virg...,Serbia pro-Kremlin disinfo campaign by Vencisl...,2019-02-24,,I00008 - Bujic
8,I00009,PhilippinesExpert,incident,President of the Philippine visited Moscow and...,2017,Russia,Philippines,https://codastory.com/disinformation/how-a-lit...,Philippines Social Media: Russian Disinformati...,2019-02-24,,I00009 - PhilippinesExpert
9,I00010,ParklandTeens,incident,IRA trolls amplified the far right messaging a...,2018,,USA,https://www.nytimes.com/2018/03/27/us/parkland...,Parkland survivors,2019-02-01,,I00010 - ParklandTeens


In [ ]:
dii = vars(amitt)['df_incidents']
dii['type'].value_counts()

In [20]:
dii[dii['type'] == 'apt']

,amitt_id,name,type,summary,year_started,attributions_seen,found_in_country,urls,Notes,when_added,found_via,longname
30,I00031,antivax,apt,,2018,Russia,World,https://ajph.aphapublications.org/doi/pdf/10.2...,,2019-02-24,OII,I00031 - antivax
32,I00033,China 50cent Army,apt,50cent Army is a CCP “tool” - “massive secret ...,2014,China,China,https://gking.harvard.edu/files/gking/files/ho...,"campaign (multiple incidents, e.g. Shanshan ri...",2019-02-24,OII,I00033 - China 50cent Army
